In [1]:
from datetime import datetime
import pandas as pd
import io
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
from docx import Document

class SendEmail:
    def __init__(self, login: str = None, password: str = None):
        self.login = login
        self.password = password

    def attach_file(self, msg: MIMEMultipart, df: pd.DataFrame, file_type: str):
        """Добавляет файл в качестве вложения"""
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        if file_type.lower() == 'excel':
            filename = f"data_export_{timestamp}.xlsx"
            
            # Создаем Excel файл в памяти
            output = io.BytesIO()
            with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
                df.to_excel(writer, sheet_name='Data', index=False)
            output.seek(0)
            
            attachment = MIMEBase('application', 'octet-stream')
            attachment.set_payload(output.read())
            
        elif file_type.lower() == 'word':
            filename = f"data_export_{timestamp}.docx"
            
            # Создаем Word документ
            doc = Document()
            doc.add_heading('Экспорт данных', 0)
            
            # Добавляем таблицу в Word
            table = doc.add_table(rows=1, cols=len(df.columns))
            table.style = 'Table Grid'
            
            # Заголовки
            for i, column in enumerate(df.columns):
                table.rows[0].cells[i].text = str(column)
            
            # Данные
            for _, row in df.iterrows():
                row_cells = table.add_row().cells
                for i, value in enumerate(row):
                    row_cells[i].text = str(value)
            
            # Сохраняем в память
            output = io.BytesIO()
            doc.save(output)
            output.seek(0)
            
            attachment = MIMEBase('application', 'octet-stream')
            attachment.set_payload(output.read())
        
        else:
            raise ValueError("Поддерживаются только 'excel' или 'word' вложения")
        
        encoders.encode_base64(attachment)
        attachment.add_header('Content-Disposition', f'attachment; filename={filename}')
        msg.attach(attachment)

    def create_html_content(self, message_text: str, df: pd.DataFrame = None):
        """Создает HTML содержимое письма"""
        
        html = f"""
        <html>
        <head>
            <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
            .info {{ background-color: #e7f3ff; padding: 15px; border-radius: 5px; }}
            </style>
        </head>
        <body>
            <div class="info">
            <p>{message_text}</p>
            </div>
        """
        
        if df is not None:
            
            html += f"""
            <p><b>Таблица данных:</b></p>
            {df.to_html(index=False, border=1, justify='center', classes='data-table')}
            """
        
        html += f"""
            <p><br>С уважением,<br>Автоматизированная система</p>
            <p><small>Письмо отправлено: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</small></p>
        </body>
        </html>
        """
        
        return html

    def send_email_smtp(self, send_to='user@gmail.com', 
                    subject="Тема письма", message_text="Текст письма",
                    df=None, json_data=None, attachment_type=None,
                    smtp_server="smtp.gmail.com", smtp_port=587,
                    cc_recipients=None, bcc_recipients=None):
        """
        Универсальная функция отправки email с различными опциями
        
        Аргументы:
        - send_to: основной получатель или список получателей
        - cc_recipients: получатели в копии (строка или список)
        - bcc_recipients: получатели в скрытой копии (строка или список)
        - subject: Тема письма
        - message_text: Текст письма
        """
        
        if not self.login or not self.password:
            raise ValueError("Логин и пароль должны быть установлены в конструкторе")
        
        # Создаем сообщение
        msg = MIMEMultipart()
        msg['From'] = self.login
        
        # Обрабатываем получателей
        if isinstance(send_to, str):
            msg['To'] = send_to
            to_recipients = [send_to]
        else:
            msg['To'] = ', '.join(send_to)
            to_recipients = send_to
        
        # Добавляем получателей в копию (CC)
        if cc_recipients:
            if isinstance(cc_recipients, str):
                msg['Cc'] = cc_recipients
                cc_list = [cc_recipients]
            else:
                msg['Cc'] = ', '.join(cc_recipients)
                cc_list = cc_recipients
        else:
            cc_list = []
        
        # BCC не добавляем в заголовки, но используем при отправке
        if bcc_recipients:
            if isinstance(bcc_recipients, str):
                bcc_list = [bcc_recipients]
            else:
                bcc_list = bcc_recipients
        else:
            bcc_list = []
        
        msg['Subject'] = subject
        
        # Обрабатываем данные
        if json_data is not None:
            if isinstance(json_data, str):
                df = pd.read_json(json_data)
            else:
                df = pd.DataFrame(json_data)
        
        # Формируем HTML содержимое
        html_content = self.create_html_content(message_text, df)
        msg.attach(MIMEText(html_content, 'html'))
        
        # Добавляем вложения если нужно
        if attachment_type and df is not None:
            self.attach_file(msg, df, attachment_type)
        

        all_recipients = to_recipients + cc_list + bcc_list
        

        try:
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls() 
                server.login(self.login, self.password)
                server.send_message(msg, to_addrs=all_recipients)
                print("Письмо успешно отправлено!")
                print(f"Получатели: {to_recipients}")
                if cc_list:
                    print(f"Копия: {cc_list}")
                if bcc_list:
                    print(f"Скрытая копия: {bcc_list}")
        except Exception as e:
            print(f"Ошибка при отправке письма: {e}")
            raise

In [3]:
from config import *
send = SendEmail(login = EMAIL, password = PASSWORD)

In [ ]:
send.send_email_smtp(
    send_to = "send@yandex.ru"
    # ,df = pd.DataFrame({'id':list(range(0, 10))})
    # ,attachment_type='word'
    ,message_text = 'Простая проверка текста'
    ,cc_recipients = ['copy_send']
    )

Письмо успешно отправлено!
Получатели: ['spasite26@yandex.ru']
Копия: ['copy_send']
